In [1]:
import os
os.chdir('../..')

In [2]:
import sys
sys.path.append('./python')

from pathlib import Path
import time
import numpy as np
import scipy.optimize
import pickle
from tqdm import tqdm

from py_diff_pd.common.common import ndarray, create_folder, rpy_to_rotation, rpy_to_rotation_gradient
from py_diff_pd.common.common import print_info, print_ok, print_error
from py_diff_pd.common.grad_check import check_gradients
from py_diff_pd.core.py_diff_pd_core import StdRealVector
from experiment.env.SorosimRountingEnv import CRoutingTendonEnv3d 
from py_diff_pd.common.renderer import PbrtRenderer
from py_diff_pd.core.py_diff_pd_core import HexMesh3d
from py_diff_pd.common.project_path import root_path


In [3]:
seed = 42
np.random.seed(seed)
folder = Path('custom_routing_tendon')
youngs_modulus = 5e5
poissons_ratio = 0.45
refinement = 11
muscle_cnt = 8
act_max = 2
env = CRoutingTendonEnv3d(seed, folder, {
    'muscle_cnt': muscle_cnt,
    'refinement': refinement,
    'youngs_modulus': youngs_modulus,
    'poissons_ratio': poissons_ratio })
deformable = env.deformable()
mesh = env.mesh
# Camera options
env._camera_pos = (0.3, -1, .25)
env._scale = 2

# Optimization parameters.
import multiprocessing
cpu_cnt = multiprocessing.cpu_count()
thread_ct = 96
print_info('Detected {:d} CPUs. Using {} of them in this run'.format(cpu_cnt, thread_ct)) 
pd_opt = { 'max_pd_iter': 500, 'max_ls_iter': 10, 'abs_tol': 1e-9, 'rel_tol': 1e-4, 'verbose': 0, 'thread_ct': thread_ct,
    'use_bfgs': 1, 'bfgs_history_size': 10 }
methods = ('pd_eigen',)
opts = (pd_opt,)

dt = 1e-2
frame_num = 10

# Initial state.
dofs = deformable.dofs()
act_dofs = deformable.act_dofs()
q0 = env.default_init_position()
v0 = np.zeros(dofs)
f0 = [np.zeros(dofs) for _ in range(frame_num)]
act_maps = env.act_maps()
u_dofs = len(act_maps)
cell_nums = env._cell_nums
node_nums = env._node_nums

assert cell_nums[0]*cell_nums[1]*cell_nums[2] == mesh.NumOfElements()
assert node_nums[0]*node_nums[1]*node_nums[2] == mesh.NumOfVertices()

def variable_to_act(x):
    act = np.ones(act_dofs)
    for i, a in enumerate(act_maps):
        act[a] = 1 - x[i]
    return act

# Get Marker
segNumber = 10

zcorrdinate = np.linspace(0,0.12,segNumber) / env._dx
xcoordinate = np.ones_like(zcorrdinate) * 0.01 /env._dx
ycoordinate = np.ones_like(zcorrdinate) * 0.02 /env._dx

py_verticeIdxs = []
for m in range(segNumber):
    if m==0:
        continue
    idx = node_nums[2]*node_nums[1]*xcoordinate[m] + node_nums[2]*ycoordinate[m] + zcorrdinate[m]
    idx = int(idx)
    py_verticeIdxs.append(idx)
py_verticeIdxs = np.array(py_verticeIdxs)

env.set_marker(py_verticeIdxs,verbose=True)


 Detected 128 CPUs. Using 96 of them in this run 
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.012727272727272728]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.025454545454545455]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.04]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.05272727272727273]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.06545454545454546]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.08]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.09272727272727273]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.10545454545454545]
Marker set to [x:0.01090909090909091,y:0.00909090909090909,z:0.12]


In [5]:
## Check Actuated element
Tendoncoordinate =(
    [(1,3),(1,7)],
    [(3,9),(7,9)],
    [(9,7),(9,3)],
    [(7,1),(3,1)]
)
tendon = Tendoncoordinate[0][0]

idx = cell_nums[2]*cell_nums[1]*tendon[0] + cell_nums[2]*tendon[1]
vertices = mesh.py_element(idx)
out = np.zeros(8*3).reshape(8,3)
assert len(vertices) == 8
for idx,vertice in enumerate(vertices):
    out[idx] = mesh.py_vertex(vertice)

In [6]:
import json

# Opening JSON file
trainInter,valInter = \
    open('python/experiment/preprocess/trainInterpolate.json'),open('python/experiment/preprocess/valInterpolate.json')

trainInterpolate, valInterpolate = json.load(trainInter), json.load(valInter)
trainInterpolate = trainInterpolate['data'], valInterpolate['data']
trainInter.close(), valInter.close()


# Set method
method,opt = methods[0], opts[0]

In [7]:
numdata = 16
# soropos = [
#     1.243147e-02,0,9.813514e-03,
#     2.485175e-02,0,9.254224e-03,
#     3.724967e-02,0,8.322633e-03,
#     4.961406e-02,0,7.019579e-03,
#     6.193380e-02,0,5.346236e-03,
#     7.419781e-02,0,3.304108e-03,
#     8.639505e-02,0,8.950347e-04,
#     9.851453e-02,0,-1.878817e-03, 
#     1.105454e-01,0,-5.014951e-03]
soropos = trainInterpolate[0][numdata]['position']
act = trainInterpolate[0][numdata]['actuation']
soropos = np.array(soropos).reshape(-1,3)


act,soropos.tolist()

([2.6604533581129344,
  6.178678304136006,
  8.704983304237876,
  1.4148667088767541],
 [[0.010514188130185818, 0.02040524166453906, 0.011329625683338534],
  [0.012049907642708774, 0.0216155720766686, 0.022508431265188377],
  [0.014586715022556939, 0.02361487930465369, 0.033387604362907754],
  [0.01809084026203724, 0.026376548548724274, 0.04382232130477341],
  [0.022515636407488646, 0.029863816437581903, 0.053673675024288686],
  [0.027802200524041462, 0.034030260422802126, 0.06281052419257777],
  [0.033880157812133015, 0.038820416756308776, 0.0711112389731956],
  [0.04066859843761213, 0.04417051882439735, 0.07846532015984603],
  [0.04807715460433965, 0.050009346009596715, 0.08477487014302922]])

In [8]:
# Check DirichletBoundaryCondition is set up properly

for i in range(node_nums[0]):
    for j in range(node_nums[1]):
        node_idx = i * node_nums[1] * node_nums[2] + j * node_nums[2]
        vx, vy, vz = mesh.py_vertex(node_idx)
        assert vz ==0


In [9]:
# Check actmaps are set up properly
acts = env.act_maps()
for act_map in acts:
    for idx in range(cell_nums[2]):
        element = act_map[idx]
        vertex = mesh.py_element(element)[0]
        xy = mesh.py_vertex(vertex)[0],mesh.py_vertex(vertex)[1]
        if idx ==0:
            standard = xy

        print(vertex)
        assert xy == standard
    for idx in range(cell_nums[2],2*cell_nums[2]):
        element = act_map[idx]
        vertex = mesh.py_element(element)[0]
        xy = mesh.py_vertex(vertex)[0],mesh.py_vertex(vertex)[1]
        if idx ==cell_nums[2]:
            standard = xy
        
        print(vertex)
        assert xy == standard

1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
6231
6232


In [13]:
data = {'actuation':act, 'position': soropos}
# act = data['actuation']
act = [0,100,0,0]
print('Actuation:', act)
act = [ele for ele in act]
act = variable_to_act(act)
markers = data['position']
print('position:', markers)
env._construct_loss_and_grad(markers)

Actuation: [0, 100, 0, 0]
position: [[0.01051419 0.02040524 0.01132963]
 [0.01204991 0.02161557 0.02250843]
 [0.01458672 0.02361488 0.0333876 ]
 [0.01809084 0.02637655 0.04382232]
 [0.02251564 0.02986382 0.05367368]
 [0.0278022  0.03403026 0.06281052]
 [0.03388016 0.03882042 0.07111124]
 [0.0406686  0.04417052 0.07846532]
 [0.04807715 0.05000935 0.08477487]]


In [14]:
vis_folder = 'test'
loss, info = env.simulate(dt, frame_num, methods[0], opts[0], q0, v0, \
    [act for _ in range(frame_num)], f0, require_grad=False, vis_folder=vis_folder)


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [15]:
from py_diff_pd.core.py_diff_pd_core import HexMesh3d
from py_diff_pd.common.project_path import root_path

vis_folder = 'test'
file_name = 'test' +'.png'

i = 10
mesh_file = str(env._folder / vis_folder / '{:04d}.bin'.format(i))

## Side view
options = {
    'file_name': file_name,
    'light_map': 'uffizi-large.exr',
    'max_depth': 2,
    'camera_pos': (1, 0.3, .25),
    'camera_lookat': (-0.1, 0, 0.15),
}

env._display_mesh(mesh_file, file_name,options)

In [17]:
from py_diff_pd.core.py_diff_pd_core import HexMesh3d
from py_diff_pd.common.project_path import root_path

vis_folder = 'test'
file_name = 'test' +'.png'

i = 10
mesh_file = str(env._folder / vis_folder / '{:04d}.bin'.format(i))

# Render.

# ## Front view
# options = {
#     'file_name': file_name,
#     'light_map': 'uffizi-large.exr',
#     'max_depth': 2,
#     'camera_pos': (0.4, -1., .25),
#     'camera_lookat': (0, .15, .15),
# }

# ## Back view
# options = {
#     'file_name': file_name,
#     'light_map': 'uffizi-large.exr',
#     'max_depth': 2,
#     'camera_pos': (0.2, 1, .55),
#     'camera_lookat': (0, -.55, -.15),
# }

## Side view
options = {
    'file_name': file_name,
    'light_map': 'uffizi-large.exr',
    'max_depth': 2,
    'camera_pos': (1, 0.3, .25),
    'camera_lookat': (-0.1, 0, 0.15),
}

renderer = PbrtRenderer(options)

mesh = HexMesh3d()
mesh.Initialize(mesh_file)
renderer.add_hex_mesh(mesh, render_voxel_edge=True, color=(.3, .7, .5), transforms=[
    ('s', env._scale),
])
renderer.add_tri_mesh(Path(root_path) / 'asset/mesh/curved_ground.obj',
    texture_img='chkbd_24_0.7', transforms=[('s', 2)])

for element in env.act_maps()[0]:
    vertices = mesh.py_element(element)
    out = np.zeros(8*3).reshape(8,3)
    assert len(vertices) == 8
    for id,vertice in enumerate(vertices):
        out[id] = mesh.py_vertex(vertice)

    marker = out.mean(0)
    
    renderer.add_shape_mesh({ 'name': 'sphere', 'center': marker, 'radius': env._dx*env._scale },
        transforms=[('s', env._scale)], color=(0.1, 0.1, 0.9))


renderer.render()

In [7]:
acts = [[10,0,0,0],[0,10,0,0],[0,0,10,0],[0,0,0,10]]
for idx,singleact in enumerate(acts):
    if True:
        act = [ele for ele in singleact]
        print("Acutation:", act)
        act = variable_to_act(act)
        vis_folder = 'test' + str(idx)
        loss, info = env.simulate(dt, frame_num, methods[0], opts[0], q0, v0, [act for _ in range(frame_num)], f0, require_grad=False, vis_folder=vis_folder)


        vis_folder = 'test' +str(idx)
        file_name = vis_folder +'.png'

        i = 10
        mesh_file = str(env._folder / vis_folder / '{:04d}.bin'.format(i))

        if idx%2==0:
            ## Back view
            options = {
                'file_name': file_name,
                'light_map': 'uffizi-large.exr',
                'max_depth': 2,
                'camera_pos': (0.2, 1, .55),
                'camera_lookat': (0, -.55, -.15),
            }
        else:
            ## Side view
            options = {
                'file_name': file_name,
                'light_map': 'uffizi-large.exr',
                'max_depth': 2,
                'camera_pos': (1, 0.3, .25),
                'camera_lookat': (-0.1, 0, 0.15),
            }
            
        renderer = PbrtRenderer(options)

        mesh = HexMesh3d()
        mesh.Initialize(mesh_file)
        renderer.add_hex_mesh(mesh, render_voxel_edge=True, color=(.3, .7, .5), transforms=[
            ('s', env._scale),
        ])
        renderer.add_tri_mesh(Path(root_path) / 'asset/mesh/curved_ground.obj',
            texture_img='chkbd_24_0.7', transforms=[('s', 2)])

        for element in env.act_maps()[idx]:
            vertices = mesh.py_element(element)
            out = np.zeros(8*3).reshape(8,3)
            assert len(vertices) == 8
            for id,vertice in enumerate(vertices):
                out[id] = mesh.py_vertex(vertice)

            marker = out.mean(0)
            
            renderer.add_shape_mesh({ 'name': 'sphere', 'center': marker, 'radius': env._dx*env._scale },
                transforms=[('s', env._scale)], color=(0.1, 0.1, 0.9))


        renderer.render()


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib